# Chapter 12 텐서플로를 사용한 사용자 정의 모델과 훈련

In [2]:
# 기본설정

# 파이썬 버전 확인
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 버전 확인
import sklearn
assert sklearn.__version__ >= "0.20"

# 공동 모듈 임포트
import numpy as np
import os

# 그래프 출력 설정
%matplotlib inline
# 위에꺼는 구버전에서 설정해야 되는거였음 설정안해도됨
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ho_chap_12"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)   # exist_ok=True 폴더가 없으면 생성

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()  # 입력없이 사용할 경우 기존에 세팅된 기본값으로 자동 레이아웃 설정
    plt.savefig(path, format=fig_extension, dpi=resolution)  # dpi값 설정을 통해 고해상도 그래프 그리기 가능

## 12.2 넘파이처럼 텐서플로 사용하기
* 텐서플로 API는 텐서를 순환시킴
* 텐서는 넘파이 ndarray와 유사. 즉, 텐서는 일반적으로 다차원 배열

### 12.2.1 텐서와 연산
* tf.constant() 함수로 텐서를 만들 수 있음

In [6]:
import tensorflow as tf
print(f"텐서플로우 버전 : {tf.__version__}")

텐서플로우 버전 : 2.6.0


In [8]:
tf.constant([[1., 2., 3.], [4., 5., 6.]])  # 2x3 행렬

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [10]:
tf.constant(42)  # 스칼라

<tf.Tensor: shape=(), dtype=int32, numpy=42>

ndarray와 마찬가지로 tf.Tensor는 크기와 데이터 타입(dtype)을 가짐

In [12]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
print("type :", t.dtype, ", shape :", t.shape)

type : <dtype: 'float32'> , shape : (2, 3)


인덱스 참조도 넘파이와 비슷하게 작동

In [14]:
t[:,1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

데이터 사이즈 변경
* tf.expend_dims(데이터, 차원)
* 데이터[..., tf.newaxis] <- 추가하고 싶은 위치에 tf.newaxis 적어주기
* reshape 또한 가능

In [15]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

모든 종류의 텐서 연산 가능
* t+10 <- tf.add(t, 10) 호출하는 것과 같음
* @ 연산은 행렬 곱셈. 이 연산은 tf.matmul() 함수를 호출하는 것과 동일

In [16]:
print(t + 10)
print(tf.square(t))
print(t @ tf.transpose(t))

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)


keras.backend를 사용하는 간단한 예

In [19]:
from tensorflow import keras
K = keras.backend
K.square(K.transpose(t)) + 10

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

### 12.2.2 텐서와 넘파이
* 텐서와 넘파이는 함께 사용하기 편리함
* 넘파이 배열로 텐서를 만들 수 있고 연산적용도 가능하고 반대도 가능

In [20]:
a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [21]:
t.numpy()  # 또는 np.array(t)

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [23]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [24]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

연산 이후에 해당 연산 자료형으로 바뀌는걸로 보임<br/>
넘파이는 64비트 정밀도를 사용하지만 텐서플로는 32비트 정밀도 사용<br/>
일반적으로 신경망은 32비트 정밀도로 충분하고 성능도 좋기 떄문에 넘파이로 텐서를 만들 때 dtype=tf.float32로 지정해야함


### 12.2.3 타입 변환
* 타입 변환은 성능을 크게 감소시킬 수 있음
* 타입이 자동으로 변환되면 사용자가 눈치채지 못할 수 있음
* 이를 방지하기 위해 텐서플로는 어떤 타입 변환도 자동으로 수행하지 않음
<br/><br/>
* 호환되지 않는 타입의 텐서로 연산을 실행하면 예외 발생
* 예를들어 실수 텐서와 정수 텐서의 연산, 32비트 실수와 64비트 실수 연산

In [26]:
tf.constant(2.) + tf.constant(40)

InvalidArgumentError: ignored

In [27]:
tf.constant(2.) + tf.constant(40., dtype=tf.float64)

InvalidArgumentError: ignored

진짜 타입 변환이 필요할 떄는 tf.cast() 함수를 사용하면 됨

In [28]:
t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

### 12.2.4 변수
* tf.Tensor는 변경이 불가능한 객체
* 따라서 tf.Variable이 필요함

In [29]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

tf.Variable은 텐서와 동일 연산 수행 가능하고 넘파이와의 호환도 베스트<br/>
But assign() 메서드를 사용하여 변숫값을 바꿀 수 있음.<br/>
V는 T를 사용하여 값을 저장함. 변수의 값을 변경 시키면 새로운 텐서가 만들어짐

### 12.2.5 다른 데이터 구조
* 자세한건 부록 F를 참고해 주세용